In [ ]:
import os
cwd=os.getcwd()
os.chdir(os.path.dirname(cwd))
print(os.getcwd())
import json
from core.data_ingestion import crunchbase
import dotenv
dotenv.load_dotenv()

from core.sql_analysis import sql_queries

%load_ext autoreload
%autoreload 2

In [3]:
import psycopg2
conn = psycopg2.connect(
    dbname="startup_database",
    user="postgres",
    password=os.getenv("DB_PASSWORD"),
    host="localhost",
    port="5432"
)
cur=conn.cursor()

In [ ]:
# get all records
cur.execute(
"""
SELECT * FROM crunchbase
"""
)
cur.fetchall()

In [ ]:
# get all records sorted by cb_rank
cur.execute("""
SELECT * FROM crunchbase
ORDER BY cb_rank ASC
LIMIT 100
""")
records=cur.fetchall()
print(len(records))
records

In [ ]:
# get records with employees>50 and cb_rank>3000
cur.execute("""
SELECT * FROM crunchbase
WHERE employees>50 AND cb_rank<3000
""")
records=cur.fetchall()
print(len(records))
records

In [ ]:
# get all records with industry=='automation'
#LEFT JOIN crunchbase, ic on crunchbase.primary_key, ic.crunchbase_primary_key

cur.execute("""
SELECT * 
FROM industry_connections ic
WHERE ic.industry_name='Communications Infrastructure';
""")
records=cur.fetchall()
print(len(records))
records

In [ ]:
# search for a specific industry in the top 200 crucnhbase companies sorted by cb_rank, with a limit of 1000 industry datapoints randomly sorted
cur.execute("""
WITH top_200 AS (
    SELECT * FROM crunchbase
    ORDER BY cb_rank ASC
    limit 200    
), industry_1000 AS (
    SELECT * FROM industry_connections
    ORDER BY random()
    LIMIT 1000
)
SELECT * FROM top_200
LEFT JOIN industry_1000 ON top_200.primary_key=industry_1000.crunchbase_primary_key
WHERE industry_name='Communications Infrastructure';

""")
records=cur.fetchall()
print(len(records))
records

In [ ]:
#CASE WHEN cb_rank<3000 THEN 1 ELSE 0 END AS cb_count
#SUM(CASE WHEN cb_rank < 3000 THEN 1 ELSE 0 END) OVER () AS total2

cur.execute("""
SELECT cb_rank,
        SUM (cb_rank) OVER (ORDER BY employees) AS total,
        SUM (CASE WHEN cb_rank < 3000 THEN 1 ELSE 0 END) OVER () AS total2,
        AVG (employees) OVER () as total3
    FROM crunchbase
WHERE cb_rank<20000
LIMIT 1
""")
records=cur.fetchall()
print(len(records))
records

In [ ]:
cur.execute("""
SELECT * from crunchbase
""")
records=cur.fetchall()
print(len(records))
records
cur.execute("ROLLBACK")